In [100]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.sandbox.regression.gmm import IV2SLS
import numpy as np
import statsmodels.api as sm
from scipy.stats import pearsonr

#!git clone https://github.com/ECMT-680-Financial-Econometrics/IV-Group-Slave-Trade.git

# Navigate to the repository directory
#%cd IV-Group-Slave-Trade

data = pd.read_csv('slave_trade_QJE_New_Data.csv')
# Preprocessing the data
data = data.drop(data.columns[[0,1]], axis=1) #Drop the first two columns

# Drop the columns that are not needed
columnstodrop = ['atlantic_distance_minimum_region_n_interaction', 'atlantic_distance_minimum_region_s_interaction', 'atlantic_distance_minimum_region_w_interaction', 'atlantic_distance_minimum_region_e_interaction', 'atlantic_distance_minimum_region_c_interaction', 'indian_distance_minimum_region_n_interaction', 'indian_distance_minimum_region_s_interaction', 'indian_distance_minimum_region_w_interaction', 'indian_distance_minimum_region_e_interaction', 'indian_distance_minimum_region_c_interaction', 'saharan_distance_minimum_region_n_interaction', 'saharan_distance_minimum_region_s_interaction', 'saharan_distance_minimum_region_w_interaction', 'saharan_distance_minimum_region_e_interaction', 'saharan_distance_minimum_region_c_interaction', 'red_sea_distance_minimum_region_n_interaction', 'red_sea_distance_minimum_region_s_interaction', 'red_sea_distance_minimum_region_w_interaction', 'red_sea_distance_minimum_region_e_interaction', 'red_sea_distance_minimum_region_c_interaction'] #Drop the first two columns
data = data.drop(columnstodrop, axis=1) 

# Preprocess the data
data = data.apply(pd.to_numeric, errors='coerce') #Convert all columns to numeric

# Scale non-binary columns for normalization
cols_to_scale = data.apply(lambda x: not pd.Series([0, 24]).isin(x).all())
data.loc[:, cols_to_scale] = scale(data.loc[:, cols_to_scale])

data = data.apply(lambda x: x.fillna(x.median()), axis=0)

# Define the dependent variable and instrumental variable
dependent_var = "ln_maddison_pcgdp2000"
instrumental_var = "ln_export_area"

# Separate the covariates and the dependent variable
X = data.drop(data.columns[0:24], axis=1)
y = data[dependent_var]
Z = data[instrumental_var]


# Perform Lasso to select important variables
lasso_y = LassoCV(cv=5, max_iter=10000, tol=0.0001, random_state=0).fit(X, y)
selected_y = X.columns[lasso_y.coef_ != 0]
print(selected_y)

lasso_z = LassoCV(cv=5, max_iter=10000, tol=0.0001, random_state=0).fit(X.drop(columns=Z.name), Z)
selected_z = X.drop(columns=Z.name).columns[(lasso_z.coef_ != 0)]
print(selected_z)

# Get the indices of selected features
selected_vars = np.unique(np.concatenate([selected_y, selected_z]))
print(selected_vars)


Index(['ln_export_area', 'colony0', 'colony3', 'colony4', 'colony5', 'colony6',
       'abs_latitude', 'rain_min', 'low_temp', 'ln_coastline_area', 'islam',
       'region_c', 'ln_avg_oil_pop', 'ln_pop_dens_1400',
       'saharan_distance_minimum', 'red_sea_distance_minimum',
       'ethnic_fractionalization', 'land_area'],
      dtype='object')
Index(['ln_export_pop', 'colony1', 'colony3', 'colony6', 'colony7', 'rain_min',
       'low_temp', 'ln_coastline_area', 'island_dum', 'islam', 'region_n',
       'region_w', 'ln_avg_gold_pop', 'ln_avg_oil_pop',
       'ln_avg_all_diamonds_pop', 'ln_pop_dens_1400',
       'saharan_distance_minimum'],
      dtype='object')
['abs_latitude' 'colony0' 'colony1' 'colony3' 'colony4' 'colony5'
 'colony6' 'colony7' 'ethnic_fractionalization' 'islam' 'island_dum'
 'land_area' 'ln_avg_all_diamonds_pop' 'ln_avg_gold_pop' 'ln_avg_oil_pop'
 'ln_coastline_area' 'ln_export_area' 'ln_export_pop' 'ln_pop_dens_1400'
 'low_temp' 'rain_min' 'red_sea_distance_minimu

In [103]:
# Create a design matrix for regression
X_selected = data[selected_vars]
X_selected = X_selected.drop(columns=[instrumental_var])

# Perform 2SLS (Two-Stage Least Squares) using the instrumental variable
# First stage: Regress the instrumental variable with selected features
first_stage = sm.OLS(Z, sm.add_constant(X_selected)).fit()

print(first_stage.summary())


                            OLS Regression Results                            
Dep. Variable:         ln_export_area   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     129.0
Date:                Wed, 24 Apr 2024   Prob (F-statistic):           1.82e-21
Time:                        05:20:21   Log-Likelihood:                 51.759
No. Observations:                  52   AIC:                            -51.52
Df Residuals:                      26   BIC:                           -0.7852
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [104]:
# Get predicted values from the first stage
Z_pred = first_stage.predict(sm.add_constant(X_selected))

# Second stage: Regress the dependent variable on the predicted values
second_stage = IV2SLS(y, sm.add_constant(X_selected), instrument=Z_pred).fit()

# Display the results
print(second_stage.summary())


                                     IV2SLS Regression Results                                      
Dep. Variable:     ln_maddison_pcgdp2000   R-squared:           -7603421372086034489988735827968.000
Model:                            IV2SLS   Adj. R-squared:     -14914403460630296863731880034304.000
Method:                        Two Stage   F-statistic:                                    4.490e-46
                           Least Squares   Prob (F-statistic):                                  1.00
Date:                   Wed, 24 Apr 2024                                                            
Time:                           05:20:29                                                            
No. Observations:                     52                                                            
Df Residuals:                         26                                                            
Df Model:                             25                                                   

c:\Python311\Lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 24
  warnings.warn('covariance of constraints does not have full '
c:\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))


In [105]:
# Step 4: Conduct IV regression using the selected instruments and controls
iv_model = IV2SLS(y, sm.add_constant(data[Z.name]), data[selected_vars]).fit()

print(iv_model.summary())

                            IV2SLS Regression Results                            
Dep. Variable:     ln_maddison_pcgdp2000   R-squared:                      -0.233
Model:                            IV2SLS   Adj. R-squared:                 -0.257
Method:                        Two Stage   F-statistic:                     12.52
                           Least Squares   Prob (F-statistic):           0.000881
Date:                   Wed, 24 Apr 2024                                         
Time:                           05:20:50                                         
No. Observations:                     52                                         
Df Residuals:                         50                                         
Df Model:                              1                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const         

In [69]:
# Post-modeling: Calculate residuals and analyze correlations
residuals_y = y - lasso_y.predict(X)
selected_vars_data = data[selected_z].copy()
selected_vars_data['Residuals'] = residuals_y

# Calculate correlations and p-values for residuals with selected variables
results = []
for var in selected_vars_data.columns[:-1]:  # Exclude 'Residuals' column
    correlation, p_value = pearsonr(selected_vars_data[var], selected_vars_data['Residuals'])
    results.append({'Variable': var, 'Correlation': correlation, 'P-value': p_value})

# Apply Bonferroni correction for multiple testing
correlation_results = pd.DataFrame(results)
num_tests = len(selected_vars_data.columns) - 1
alpha = 0.05 / num_tests
correlation_results['Significant'] = correlation_results['P-value'] < alpha

# Output the results of the analysis
print(correlation_results)

                                         Variable  Correlation   P-value  \
0                                   ln_export_pop    -0.057831  0.683838   
1                                         colony1     0.059027  0.677653   
2                                         colony3    -0.061679  0.664013   
3                                         colony6     0.016497  0.907590   
4                                         colony7    -0.061692  0.663951   
5                                        rain_min    -0.061693  0.663942   
6                                        low_temp    -0.051138  0.718821   
7                               ln_coastline_area     0.061694  0.663939   
8                                      island_dum     0.042442  0.765132   
9                                           islam    -0.061690  0.663960   
10                                ln_avg_gold_pop     0.015288  0.914336   
11                               ln_pop_dens_1400    -0.050958  0.719773   
12   indian_